<a href="https://colab.research.google.com/github/christinajoslin/multilingual-sentiment-classifier/blob/main/Multilingual_Amazon_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install the Hugging Face `datasets` library if not already installed
#!pip install datasets

# Import the Dataset class for working with datasets in Hugging Face
from datasets import Dataset

# Import pandas for data manipulation and analysis
import pandas as pd

# Import numpy for numerical computations
import numpy as np

# Import the tokenizer and model for sequence classification from the Hugging Face Transformers library
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Import Trainer and TrainingArguments for fine-tuning and training models
from transformers import Trainer, TrainingArguments

# Import accuracy_score and precision_recall_fscore_support for custom evaluation metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Import EarlyStoppingCallback for stopping training early if no improvement in validation loss
from transformers import EarlyStoppingCallback


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
train_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Multilingual Amazon Reviews/train.csv")
test_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Multilingual Amazon Reviews/test.csv")
dev_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Multilingual Amazon Reviews/validation.csv")

In [49]:
# Print the number of examples in the training set
# Training set has 1.2 million examples (95.2% of the total data)
print(f"Training Examples: {train_df.shape[0]}")

# Print the number of examples in the testing set
# Testing set has 30,000 examples (2.4% of the total data)
print(f"Testing Examples: {test_df.shape[0]}")

# Print the number of examples in the development (validation) set
# Development set has 30,000 examples (2.4% of the total data)
print(f"Development Examples: {dev_df.shape[0]}")

# Calculate the total number of examples across all datasets
total_examples = train_df.shape[0] + test_df.shape[0] + dev_df.shape[0]
print(f"Total Examples: {total_examples}")  # Should equal 1.26 million in this case

# Calculate the percentage of examples in the test set
test_set_percentage = test_df.shape[0] / total_examples * 100

# Calculate the percentage of examples in the development set
dev_set_percentage = dev_df.shape[0] / total_examples * 100

# Calculate the percentage of examples in the training set
train_set_percentage = train_df.shape[0] / total_examples * 100

# Print the calculated percentages for each dataset
# Should confirm the split as approximately 95.2% training, 2.4% testing, and 2.4% development
print(f"Test Set Percentage: {test_set_percentage:.1f}%")
print(f"Development Set Percentage: {dev_set_percentage:.1f}%")
print(f"Training Set Percentage: {train_set_percentage:.1f}%")


Training Examples: 1200000
Testing Examples: 30000
Development Examples: 30000
Total Examples: 1260000
Test Set Percentage: 2.4%
Development Set Percentage: 2.4%
Training Set Percentage: 95.2%


In [50]:
train_df.head()

,Unnamed: 0,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,0,de_0203609,product_de_0865382,reviewer_de_0267719,1,Armband ist leider nach 1 Jahr kaputt gegangen,Leider nach 1 Jahr kaputt,de,sports
1,1,de_0559494,product_de_0678997,reviewer_de_0783625,1,In der Lieferung war nur Ein Akku!,EINS statt ZWEI Akkus!!!,de,home_improvement
2,2,de_0238777,product_de_0372235,reviewer_de_0911426,1,"Ein Stern, weil gar keine geht nicht. Es hande...",Achtung Abzocke,de,drugstore
3,3,de_0477884,product_de_0719501,reviewer_de_0836478,1,"Dachte, das wären einfach etwas festere Binden...",Zu viel des Guten,de,drugstore
4,4,de_0270868,product_de_0022613,reviewer_de_0736276,1,Meine Kinder haben kaum damit gespielt und nac...,Qualität sehr schlecht,de,toy


In [51]:
# Define the target languages to include in the filtered datasets
target_languages = ['en', 'es', 'fr', 'de']  # English, Spanish, French, German

# Filter the training dataset to include only examples in the target languages
# Result: 800,000 examples remain after filtering
train_df = train_df[train_df['language'].isin(target_languages)]

# Filter the development (validation) dataset to include only examples in the target languages
# Result: 20,000 examples remain after filtering
dev_df = dev_df[dev_df['language'].isin(target_languages)]

# Filter the testing dataset to include only examples in the target languages
# Result: 20,000 examples remain after filtering
test_df = test_df[test_df['language'].isin(target_languages)]


In [52]:
train_df = train_df[['review_body', 'stars']]
test_df = test_df[['review_body', 'stars']]
dev_df = dev_df[['review_body', 'stars']]

In [53]:
def convert_to_polarity(stars):
    '''
    Converts star ratings into sentiment polarity categories.
    Input:
      - stars: An integer representing the star rating (1 to 5).
    Output:
      - 0 for negative sentiment (1 or 2 stars)
      - 1 for neutral sentiment (3 stars)
      - 2 for positive sentiment (4 or 5 stars)
    '''
    if stars in [1, 2]:
        return 0  # Negative sentiment
    elif stars == 3:
        return 1  # Neutral sentiment
    elif stars in [4, 5]:
        return 2  # Positive sentiment

# Apply the `convert_to_polarity` function to the 'stars' column in each dataset
# This creates a new 'polarity' column representing sentiment
train_df['polarity'] = train_df['stars'].apply(convert_to_polarity)  # Polarity for training data
test_df['polarity'] = test_df['stars'].apply(convert_to_polarity)   # Polarity for test data
dev_df['polarity'] = dev_df['stars'].apply(convert_to_polarity)     # Polarity for validation data


In [54]:
train_df.head()

,review_body,stars,polarity
0,Armband ist leider nach 1 Jahr kaputt gegangen,1,0
1,In der Lieferung war nur Ein Akku!,1,0
2,"Ein Stern, weil gar keine geht nicht. Es hande...",1,0
3,"Dachte, das wären einfach etwas festere Binden...",1,0
4,Meine Kinder haben kaum damit gespielt und nac...,1,0


In [55]:
# Since the datasets are incredibly large, and due to processing and memory limitations,
# I will only use a subset of the data. The subset will still preserve the proportion of
# samples with respect to the star ratings to maintain dataset balance.

# Define the total size of the new subset
new_total_size = 84000  # Total number of samples to use across train, dev, and test subsets

# Define the proportions for each subset (70% training, 20% development, 10% testing)
train_size = int(new_total_size * 0.70)  # 70% of the total size
dev_size = int(new_total_size * 0.20)    # 20% of the total size
test_size = int(new_total_size * 0.10)   # 10% of the total size

# Function to create a stratified sample
def stratified_sample(df, target_col, sample_size):
    """
    Create a stratified sample from the dataset.
    Ensures the proportion of each category in the target column (e.g., star ratings)
    is preserved in the sample.

    Parameters:
    - df (DataFrame): The input dataset to sample from
    - target_col (str): The column name representing categories (e.g., 'stars')
    - sample_size (int): The total number of samples to extract from the dataset

    Returns:
    - DataFrame: A stratified sample of the dataset with preserved proportions
    """
    # Group by the target column and sample from each group based on its proportion
    stratified_sample = df.groupby(target_col, group_keys=False).apply(
        lambda x: x.sample(n=min(len(x), int(sample_size * len(x) / len(df))), random_state=42)
    )
    return stratified_sample

# Perform stratified sampling for training, development, and test subsets
train_subset = stratified_sample(train_df, target_col='stars', sample_size=train_size)
dev_subset = stratified_sample(dev_df, target_col='stars', sample_size=dev_size)
test_subset = stratified_sample(test_df, target_col='stars', sample_size=test_size)

# Verify that the star rating proportions are preserved in each subset
print("Train Subset Distribution:")
print(train_subset['stars'].value_counts(normalize=True))  # Prints proportions of star ratings in training data

print("Dev Subset Distribution:")
print(dev_subset['stars'].value_counts(normalize=True))    # Prints proportions of star ratings in development data

print("Test Subset Distribution:")
print(test_subset['stars'].value_counts(normalize=True))   # Prints proportions of star ratings in testing data


Train Subset Distribution:
stars
1    0.2
2    0.2
3    0.2
4    0.2
5    0.2
Name: proportion, dtype: float64
Dev Subset Distribution:
stars
1    0.2
2    0.2
3    0.2
4    0.2
5    0.2
Name: proportion, dtype: float64
Test Subset Distribution:
stars
1    0.2
2    0.2
3    0.2
4    0.2
5    0.2
Name: proportion, dtype: float64


<ipython-input-55-cfc227ddcf40>:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratified_sample = df.groupby(target_col, group_keys=False).apply(
<ipython-input-55-cfc227ddcf40>:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratified_sample = df.groupby(target_col, group_keys=False).apply(
<ipython-input-55-cfc227ddcf40>:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping column

In [56]:
# Convert the stratified subsets into Hugging Face Dataset objects
# Include only the 'review_body' (text) and 'polarity' (label) columns
train_set = Dataset.from_pandas(train_subset[['review_body', 'polarity']])  # Training dataset
test_set = Dataset.from_pandas(test_subset[['review_body', 'polarity']])    # Testing dataset
dev_set = Dataset.from_pandas(dev_subset[['review_body', 'polarity']])      # Development (validation) dataset

# Define a preprocessing function to format the data for model input
def preprocess_function(example):
    """
    Preprocesses the input examples to create a text-label mapping.

    Parameters:
    - example (dict): A dictionary containing 'review_body' (text) and 'polarity' (label).

    Returns:
    - dict: A dictionary with the processed text and label ready for training.
      Keys:
        - "text": The review text (from 'review_body')
        - "label": The polarity label (from 'polarity')
    """
    return {
        "text": example["review_body"],  # Map the review text
        "label": example["polarity"]    # Map the polarity label
    }

# Preprocess the datasets by applying the preprocessing function
# The `map` function applies the `preprocess_function` to all examples in the dataset
train_set = train_set.map(preprocess_function, batched=True)  # Preprocess training dataset
test_set = test_set.map(preprocess_function, batched=True)    # Preprocess testing dataset
dev_set = dev_set.map(preprocess_function, batched=True)      # Preprocess development dataset


Map:   0%|          | 0/58795 [00:00<?, ? examples/s]

Map:   0%|          | 0/8400 [00:00<?, ? examples/s]

Map:   0%|          | 0/16800 [00:00<?, ? examples/s]

In [57]:
# Specify the pre-trained model to be used for sentiment analysis
# Using a multilingual BERT model fine-tuned for sentiment classification
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

# Load the tokenizer associated with the specified model
# This tokenizer is used to tokenize the text inputs into the format required by the model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the pre-trained model with the specified number of labels
# - `num_labels=3`: Model output corresponds to three sentiment classes (negative, neutral, positive)
# - `ignore_mismatched_sizes=True`: Allows adjustment if the model weights or architecture do not match perfectly
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    ignore_mismatched_sizes=True
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
# Define a function to tokenize the dataset
# The function uses the tokenizer to convert text inputs into tokenized format required by the model
def tokenize_function(examples):
    """
    Tokenizes the input examples for the model.

    Parameters:
    - examples (dict): A dictionary containing the "text" field to be tokenized.

    Returns:
    - dict: A dictionary with tokenized input features, including:
      - input_ids: Tokenized numerical representation of the text
      - attention_mask: Indicates which tokens are padding
    """
    # Tokenize the text with the following parameters:
    # - padding="max_length": Ensures all sequences are padded to the same length (128 tokens here)
    # - truncation=True: Truncates text that exceeds the maximum length
    # - max_length=128: Specifies the maximum token length for each input
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# Apply the tokenize function to the training dataset
# `batched=True` allows tokenization of multiple examples at once for faster processing
tokenized_train = train_set.map(tokenize_function, batched=True)

# Apply the tokenize function to the test dataset
tokenized_test = test_set.map(tokenize_function, batched=True)

# Apply the tokenize function to the development (validation) dataset
tokenized_dev = dev_set.map(tokenize_function, batched=True)

Map:   0%|          | 0/58795 [00:00<?, ? examples/s]

Map:   0%|          | 0/8400 [00:00<?, ? examples/s]

Map:   0%|          | 0/16800 [00:00<?, ? examples/s]

In [59]:
from google.colab import output

# JavaScript snippet to keep the Google Colab runtime active
# - Executes a function `KeepClicking` every 60 seconds (60000 ms)
# - Simulates clicking the "Connect" button in Colab to prevent the runtime from disconnecting due to inactivity
output.eval_js('''
function KeepClicking() {
    console.log("Clicked on the 'Connect' button");  // Log the click action to the console
    document.querySelector("#top-toolbar > colab-connect-button")
        .shadowRoot.querySelector("#connect").click();  // Find and click the "Connect" button
}
setInterval(KeepClicking, 60000);  // Repeat every 60 seconds
''')

# Define the training arguments for fine-tuning the model
training_args = TrainingArguments(
    output_dir="./results",  # Directory to save the model checkpoints and results
    evaluation_strategy="epoch",  # Perform evaluation at the end of every epoch
    save_strategy="epoch",  # Save model checkpoints at the end of every epoch
    learning_rate=1e-4,  # Initial learning rate for the optimizer
    logging_steps=100,  # Log training progress every 100 steps
    save_steps=100,  # Save model checkpoint every 100 steps
    per_device_train_batch_size=64,  # Training batch size per GPU/TPU/CPU
    per_device_eval_batch_size=64,  # Evaluation batch size per GPU/TPU/CPU
    gradient_accumulation_steps=8,  # Accumulate gradients over 8 steps before updating model weights
    num_train_epochs=7,  # Number of training epochs
    weight_decay=0.05,  # Weight decay for regularization
    load_best_model_at_end=True,  # Load the best model (based on evaluation metric) after training
    metric_for_best_model="accuracy",  # Metric used to determine the best model
    report_to="none"  # Disable reporting to external tools (e.g., WandB, TensorBoard)
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [60]:
# Define the evaluation metrics for the model's performance

def compute_metrics(eval_pred):
    """
    Computes evaluation metrics for the model's predictions.

    Args:
        eval_pred (tuple): A tuple containing:
            - logits (ndarray): The predicted raw scores (logits) from the model.
            - labels (ndarray): The true labels for the evaluation dataset.

    Returns:
        dict: A dictionary containing accuracy, F1 score, precision, and recall.
    """
    # Unpack the logits and true labels from the evaluation predictions
    logits, labels = eval_pred

    # Convert logits to predicted class indices by selecting the maximum score
    predictions = np.argmax(logits, axis=-1)

    # Compute precision, recall, and F1-score using scikit-learn
    # 'average="weighted"' ensures metrics account for label imbalance
    # 'zero_division=1' avoids division errors when precision/recall are undefined
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted', zero_division=1
    )

    # Compute accuracy as the proportion of correct predictions
    acc = accuracy_score(labels, predictions)

    # Return a dictionary containing all calculated metrics
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}



In [61]:
# Initialize Trainer
trainer = Trainer(
    model=model,  # The pre-trained model to be fine-tuned or trained
    args=training_args,  # Configuration for the training process (e.g., batch size, learning rate)
    train_dataset=tokenized_train,  # Tokenized training dataset
    eval_dataset=tokenized_dev,  # Tokenized validation dataset for evaluation
    tokenizer=tokenizer,  # Tokenizer used for processing input text data
    callbacks=[  # List of callbacks to monitor and adjust training
        EarlyStoppingCallback(early_stopping_patience=1)  # Stops training early if no improvement after 1 patience epoch
    ],
    compute_metrics=compute_metrics  # Function to calculate evaluation metrics (e.g., accuracy, F1-score)
)


<ipython-input-61-e47459522210>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [62]:
#import torch

# Empty the GPU cache to free up memory before training.
# This can be helpful to avoid out-of-memory errors, especially when working with large models or datasets.
#torch.cuda.empty_cache()

In [63]:
trainer.train() #takes approx. 1 hour to train on a T4 GPU

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,4.811600,0.565809,0.757738,0.749776,0.747953,0.757738
1,4.015700,0.563478,0.764464,0.752092,0.746606,0.764464
2,3.403600,0.648475,0.754167,0.751430,0.750997,0.754167


TrainOutput(global_step=342, training_loss=3.969965750711006, metrics={'train_runtime': 3560.1255, 'train_samples_per_second': 115.604, 'train_steps_per_second': 0.224, 'total_flos': 1.155594130661376e+16, 'train_loss': 3.969965750711006, 'epoch': 2.9923830250272037})

In [64]:
#Evaluate accuracy of training and validation sets

train_results = trainer.evaluate(tokenized_train)
print(f"Train Accuracy: {train_results['eval_accuracy']:.4f}")  # Print train accuracy
print(f"Train F1 Score: {train_results['eval_f1']:.4f}")     # Print train F1 score

dev_results = trainer.evaluate(tokenized_dev)
print(f"Validation Accuracy: {dev_results['eval_accuracy']:.4f}")  # Print validation accuracy
print(f"Validation F1 Score: {dev_results['eval_f1']:.4f}")     # Print validation F1 score


Train Accuracy: 0.8518
Train F1 Score: 0.8425
Validation Accuracy: 0.7645
Validation F1 Score: 0.7521


In [65]:
#Evaluate accuracy of test set
test_results = trainer.evaluate(tokenized_test)
print(f"Test Accuracy: {test_results['eval_accuracy']:.4f}")  # Print test accuracy
print(f"Test F1 Score: {test_results['eval_f1']:.4f}")     # Print test F1 score

Test Accuracy: 0.7663
Test F1 Score: 0.7556
